# Лабораторна робота №1

Бригада №3, КА-61м
- Одобеску Владислав
- Фатенко Владислав
- Шевченко Тарас

## Завдання

Розв'язати задачу методом аналізу ієрархій, використовуючи наступні методи:
- ідеальний синтез
- групове врахування бінарних відношень переваг альтернатив (ГВБВПА) з ідеальним синтезом

План виконання:

#### Частина І
1. зобразити графічно ієрархію, задану умовою задачі
2. розв'язати задачу вказаними методами синтезу
3. порівняти результати, отримані різними методами; якщо метод ГВБВПА виконувався для того ж типу синтезу, вказати, чи відбувся реверс рангів третього типу

#### Частина ІІ
1. додати (n+1)-у альтернативу до початкового набору альтернатив з умови задачі і згенерувати її випадкову вагу за кожним із критеріїв; пронормувати ваги за кожним із критеріїв
2. розрахувати нові глобальні ваги заданими методами синтезу, порівняти порядок рангів старих альтернатив із отриманим в п.3 [ч.1][1]; перевірити, чи спостерігається явище реверсу рангів першого або другого типу.
[1]: #Частина-І

Розглядається задача №3:

> Нехай інвестор оцінює акції деякої компаній і хоче спрогнозувати яким буде розподіл ймовірностей зміни ціни на них. Він розглядає наступні можливі варіанти зміни ціни: впаде на 20%, впаде на 10%, залишиться незмінною, зросте на 10%. Оцінки кожного варіанту за результатами проведених фундаментального та технічного аналізу, а також аналізу циклів наведені в таблиці. Результати, отримані різними методами, є для інвестора однаково важливими.

<table>
<tr>
<th>Варіанти зміни ціни</th>
<th>Впаде на 20%</th>
<th>Впаде на 10%</th>
<th>Залишиться незмінною</th>
<th>Зросте на 10%</th>
</tr>
<tr>
<td>Впаде на 20%</td>
<td>-</td><td>-</td><td>-</td><td>-</td>
</tr>
<tr>
<td>Впаде на 10%</td>
<td>ненабагато перевищує</td><td>-</td><td>-</td><td>-</td>
</tr>
<tr>
<td>Залишиться незмінною</td>
<td>ненабагато перевищує</td><td>ненабагато перевищує</td><td>-</td><td>-</td>
</tr>
<tr>
<td>Зросте на 10%</td>
<td>суттєво перевищує</td><td>суттєво перевищує</td><td>ненабагато перевищує</td><td>-</td>
</tr>
</table>

## Виконання роботи

### Імпорт бібліотек

In [1]:
%pylab inline
import seaborn as sns
import scipy as sp
import pandas as pd
from scipy import stats
from IPython.display import display

Populating the interactive namespace from numpy and matplotlib


### Підготовка програмних функцій

Для визначення ваг альтернатив опісля застосування методу ГВБВПА будемо використовувати EM-метод.

In [2]:
def em(mat):
    """
    EM-method to determine alternatives' weights.
    """
    values, vectors = linalg.eig(mat)
    v = vectors[:, np.argmax(abs(values))]
    assert (v.imag < 1e-6).all(), "eigenvector has complex coordinates: %s" % v.imag
    v = v.real
    if (v < 0).all():
        v = - v
    assert (v >= 0).all(), "eigenvector has negative coordinates: %s" % v
    return v / linalg.norm(v, ord=1)

Нехай задані:
- множина альтернатив $A=\{a_i| \; i=1,\dots,n\}$;
- множина критеріїв (цілей) $C=\{C_j| \; j=1,\dots,m\}$;
- ваги критеріїв $W^C=\{w_j^C| \; j=1,\dots,m\}$;
- ваги альтернатив відносно критеріїв $W=\{w_{ij}| \; i=1,\dots,n; \; j=1,\dots,m\}$, $w_{ij}$ - вага $i$-ої альтернативи відносно $j$-ого критерію.

Необхідно визначити глобальні ваги альтернатив $W^{glob}=\{w_i^{glob}| \; i=1,\dots,n\}$.

Тоді процедура ідеального синтезу визначається наступним чином:
$$r_{ij} = \frac{w_{ij}}{\max\limits_{i=\overline{1,n}}{w_{ij}}} \quad v_i^{glob}=\sum\limits_{j=1}^m r_{ij} w_j^C$$
$$w_j^{glob} = \frac{v_i^{glob}}{\sum\limits_{i=1}^n v_{i}^{glob}}$$

In [3]:
def perfect_synthesis(w, wc):
    """
    Performs perfect synthesis on given weight matrix [[w11,...,wn1],...[w1m,...,wnm]]
    and criterion weight vector [wc1,...,wcm].
    """
    r = w.T / w.max(axis=1)
    v = sum(r * wc, axis=1)
    return v / linalg.norm(v, ord=1)

Метод ГВБВПА з ідеальним синтезом полягає в застосуванні ідеального синтезу для кожної пари альтернатив окремо, отримуючи пари ваг $w_i^{ij}, w_j^{ij},\; i,j=1,\dots,n$. Далі за допомогою цих ваг будується МПП $P=\left(\frac{w_i^{ij} }{ w_j^{ij}}\right)_{i,j=\overline{1,n}}$.
До отриманої матриці попарних порівнянь застосовується один із методів для одиничних критеріїв (в нашому випадку буде використовуватися EM, як вже було зазначено).

In [4]:
def gcbraa(w, wc, synt):
    
    """
    Performs GCBRAA synthesis on given weight matrix [[w11,...,wn1],...[w1m,...,wnm]]
    and criterion weight vector [wc1,...,wcm].
    Using synthesis method synt.
    """
    assert (abs(w.sum(axis=1) - 1) < 1e-3).all(), "given alternative weights are not normalized: %s" % w.sum(axis=1)
    assert abs(wc.sum() - 1) < 1e-3             , "given criterion weights are not normalized: %s" % wc.sum()
    prm = identity(w.shape[1])
    alt_count = w.shape[1]
    for i in range(alt_count - 1):
        for j in range(i + 1, alt_count):
            weights = synt(w[:,(i,j)], wc)
            prm[i, j], prm[j, i] = weights[0] / weights[1], weights[1] / weights[0]
    return em(prm)

 ### Частина I

![alt](diah_hierarchy.jpg)

Введемо вихідні дані відповідно до умови задачі. Маємо 3 критерії: фундаментальний аналіз, технічний аналіз, аналіз циклів. Кожен з цих методів має однакову важливість для інвестора, тому $W^C=\{\frac13, \frac13, \frac13\}$.

In [5]:
w = array([[0.1, 0.2, 0.3, 0.4],
           [0.1, 0.2, 0.3, 0.4],
           [0.15, 0.25, 0.35, 0.25]])
wc = ones(3) / 3 # all criterions have same importance
criterions = ['fundamental analysis', 'technical analysis', 'cycle analysis']
alternatives = ['20% drop', '10% drop', 'stable', '10% rise']
display(pd.DataFrame(vstack((w.T, wc)), columns=criterions, index=alternatives+['$w^C$']))

,fundamental analysis,technical analysis,cycle analysis
20% drop,0.100000,0.100000,0.150000
10% drop,0.200000,0.200000,0.250000
stable,0.300000,0.300000,0.350000
10% rise,0.400000,0.400000,0.250000
$w^C$,0.333333,0.333333,0.333333


Застосуємо метод ідеального синтезу:

In [6]:
display(pd.DataFrame(perfect_synthesis(w, wc)[newaxis,:], columns=alternatives))

,20% drop,10% drop,stable,10% rise
0,0.118182,0.218182,0.318182,0.345455


Застосуємо метод ГВБВПА з ідеальним синтезом:

In [7]:
display(pd.DataFrame(gcbraa(w, wc, perfect_synthesis)[newaxis,:], columns=alternatives))

,20% drop,10% drop,stable,10% rise
0,0.119754,0.222628,0.321256,0.336362


Як видно, альтернативи 'не зміниться' та 'збільшиться на 10%' стали ближчими, але реверс рангів 3-го роду не відбувся, адже порядок альтернатив відсортованих за ймовірністю не змінився. Порядок альтернатив за зростанням ваги: $A_1, A_2, A_3, A_4$.

### Частина II

Додамо нову альтернативу 'зросте на 20%' та дамо їй випадкові ваги за всіма критеріями, після цього нормуємо матрицю ваг.

In [8]:
alternatives_mod = alternatives + ['20% rise']
w_mod = hstack((w, random.uniform(0,1,size=3)[:, newaxis]))
w_mod /= w_mod.sum(axis=1)[:, newaxis]
display(pd.DataFrame(vstack((w_mod.T, wc)), columns=criterions, index=alternatives_mod+['$w^C$']))

,fundamental analysis,technical analysis,cycle analysis
20% drop,0.057600,0.052212,0.078537
10% drop,0.115200,0.104424,0.130895
stable,0.172800,0.156636,0.183253
10% rise,0.230400,0.208848,0.130895
20% rise,0.424000,0.477881,0.476421
$w^C$,0.333333,0.333333,0.333333


Застосуємо метод ідеального синтезу:

In [9]:
display(pd.DataFrame(perfect_synthesis(w_mod, wc)[newaxis,:], columns=alternatives_mod))

,20% drop,10% drop,stable,10% rise,20% rise
0,0.062588,0.116787,0.170985,0.191628,0.458012


Застосуємо метод ГВБВПА з ідеальним синтезом:

In [10]:
display(pd.DataFrame(gcbraa(w_mod, wc, perfect_synthesis)[newaxis,:], columns=alternatives_mod))

,20% drop,10% drop,stable,10% rise,20% rise
0,0.064221,0.119467,0.172863,0.183549,0.4599


Як видно, реверс рангів першого, а тим паче другого роду, відсутній.

### Висновок

Під час виконання лабораторної роботи було розв'язано задачу методом аналізу ієрархій, а саме за допомогою методу ідеального синтезу та методу ГВБВПА з ідеальним синтезом.
Було проведено тестування на існування реверсу рангів 1-3 роду, результат негативний.
Остаточний порядок альтернатив: $A_1 \prec A_2 \prec A_3 \prec A_4$.

|  Код  |          Ім'я         |
|:-----:|:---------------------:|
| $A_1$ |      Впаде на 20%     |
| $A_2$ |      Впаде на 10%     |
| $A_3$ | Залишиться не змінною |
| $A_4$ |     Зросте на 10%     |